# Teleoperation

In this example we'll control the Jetbot remotely with a gamepad controller connected to our web browser machine.

### Create gamepad controller

The first thing we want to do is create an instance of the ``Controller`` widget, which we'll use to drive our robot.
The ``Controller`` widget takes a ``index`` parameter, which specifies the number of the controller.  This is useful in case you
have multiple controllers attached, or some gamepads *appear* as multiple controllers.  To determine the index
of the controller you're using,

1. Visit [http://html5gamepad.com](http://html5gamepad.com).  
2. Press buttons on the gamepad you're using
3. Remember the ``index`` of the gamepad that is responding to the button presses

Next, we'll create and display our controller using that index.

In [1]:
import ipywidgets.widgets as widgets
import traitlets
from jetracer.nvidia_racecar import NvidiaRacecar
import time 
controller = widgets.Controller(index=0)
display(controller)

WARNNIG: Jetson.GPIO library has not been verified with this carrier board,


Controller()

Even if the index is correct, you may see the text ``Connect gamepad and press any button``.  That's because the gamepad hasn't
registered with this notebook yet.  Press a button and you should see the gamepad widget appear above.

### Connect gamepad controller to robot motors

Now, even though we've connected our gamepad, we haven't yet attached the controls to our robot!  The first, and most simple control
we want to attach is the motor control.  We'll connect that to the left and right vertical axes using the ``dlink`` function.  The
``dlink`` function, unlike the ``link`` function, allows us to attach a transform between the ``source`` and ``target``.  Because
the controller axes are flipped from what we think is intuitive for the motor control, we'll use a small *lambda* function to
negate the value.

> WARNING: This next cell will move the robot if you touch the gamepad controller axes!

In [2]:
car = NvidiaRacecar()

In [3]:
# COMPLETE NUCLEAR OPTION + PROPER PRIMING - ALL IN ONE CELL
import ipywidgets.widgets as widgets
import traitlets
from jetracer.nvidia_racecar import NvidiaRacecar
import time

print("💥 NUCLEAR OPTION WITH PROPER PRIMING")

# Step 1: Fresh car
car = NvidiaRacecar()
car.steering_gain = -0.8
car.steering_offset = 0.16
car.throttle_gain = 0.2                           # Start with target
car.throttle = 0
print("✅ Fresh car created")

# Step 2: Proper priming sequence
print("🔄 PROPER PRIMING SEQUENCE")

# Increase gain for priming
car.throttle_gain = 0.4
print(f"   Priming gain: {car.throttle_gain}")
time.sleep(0.2)

# Prime forward
print("   Priming forward...")
car.throttle = 0.6
time.sleep(0.4)
car.throttle = 0

# Prime reverse  
print("   Priming reverse...")
car.throttle = -0.6
time.sleep(0.4)
car.throttle = 0

# Mixed sequence
print("   Mixed sequence...")
car.throttle = 0.8
time.sleep(0.3)
car.throttle = -0.8
time.sleep(0.3)
car.throttle = 0

# Set final working gain
car.throttle_gain = 0.2                           # Back to working value
print(f"   Final gain: {car.throttle_gain}")
time.sleep(0.3)
car.throttle = 0

print("✅ Priming complete!")



💥 NUCLEAR OPTION WITH PROPER PRIMING
✅ Fresh car created
🔄 PROPER PRIMING SEQUENCE
   Priming gain: 0.4
   Priming forward...
   Priming reverse...
   Mixed sequence...
   Final gain: 0.2
✅ Priming complete!


In [4]:
# CLEAN SMART THROTTLE SETUP
import traitlets

# Steering link (unchanged)
steering_link = traitlets.dlink(
    (controller.axes[0], 'value'), 
    (car, 'steering'), 
    transform=lambda x: x
)

# Initialize tracking variables
last_throttle = 0
forward_was_used = False

def smart_throttle_transform(controller_value):
    global last_throttle, forward_was_used
    
    # Get current throttle value
    current_throttle = max(-1.0, min(1.0, -controller_value))
    
    # Detect when we stop after going forward
    if last_throttle > 0.2 and abs(current_throttle) < 0.1:
        forward_was_used = True
        
    # If trying reverse after forward, refresh first
    if forward_was_used and current_throttle < -0.1:
        print("🔄 Auto-refreshing reverse...")
        
        # Quick reverse refresh
        old_gain = car.throttle_gain
        car.throttle_gain = 0.6
        car.throttle = -0.7
        time.sleep(0.2)
        car.throttle = 0
        time.sleep(0.1)
        car.throttle_gain = old_gain
        
        forward_was_used = False
        print("✅ Reverse ready!")
    
    # Update state
    last_throttle = current_throttle
    
    return current_throttle

# Create smart throttle link
throttle_link = traitlets.dlink(
    (controller.axes[3], 'value'), 
    (car, 'throttle'), 
    transform=smart_throttle_transform
)

print("✅ Smart throttle with auto reverse refresh active!")
print("📊 Forward → stop → reverse will auto-refresh")

✅ Smart throttle with auto reverse refresh active!
📊 Forward → stop → reverse will auto-refresh


In [5]:
# ULTRA-STRONG REVERSE PRIMING - Replace previous priming
print("⚡ ULTRA-STRONG REVERSE PRIMING")

# Step 1: Maximum gain for maximum power
old_gain = car.throttle_gain
car.throttle_gain = 0.8  # Very high gain
print(f"   Ultra-high gain: {car.throttle_gain}")

# Step 2: Maximum reverse shock
print("   Maximum reverse shock...")
car.throttle = -1.0  # Full reverse
time.sleep(1.0)      # Longer duration
car.throttle = 0
time.sleep(0.5)

# Step 3: Repeated reverse shocks
for i in range(3):
    print(f"   Reverse shock {i+1}/3...")
    car.throttle = -1.0
    time.sleep(0.8)
    car.throttle = 0
    time.sleep(0.3)

# Step 4: Aggressive alternating pattern
print("   Aggressive alternating...")
for i in range(5):
    car.throttle = 1.0   # Full forward
    time.sleep(0.2)
    car.throttle = -1.0  # Full reverse
    time.sleep(0.2)
car.throttle = 0

# Step 5: Final reverse test with high gain
print("   Final reverse test with high gain...")
car.throttle = -0.8
time.sleep(1.0)
car.throttle = 0

# Step 6: Restore working gain
car.throttle_gain = old_gain  # Back to 0.2
print(f"✅ Gain restored to: {car.throttle_gain}")
print("🧪 Try controller reverse now!")

⚡ ULTRA-STRONG REVERSE PRIMING
   Ultra-high gain: 0.8
   Maximum reverse shock...
   Reverse shock 1/3...
   Reverse shock 2/3...
   Reverse shock 3/3...
   Aggressive alternating...
   Final reverse test with high gain...
✅ Gain restored to: 0.2
🧪 Try controller reverse now!


Awesome! Our robot should now respond to our gamepad controller movements.  Now we want to view the live video feed from the camera!

In [ ]:
#car = NvidiaRacecar()
#car.steering_gain = -0.8
#car.steering_offset = 0.16
​
# BIDIRECTIONAL PRIMING
#car.throttle_gain = 0.4  # Higher for priming
​
# Prime FORWARD
#car.throttle = 0.2
#time.sleep(0.3)
#car.throttle = 0
​
# Prime REVERSE (the missing piece!)
#car.throttle = -0.2  # Negative for reverse
#time.sleep(0.3)
#car.throttle = 0
#time.sleep(0.2)
​
# Set working gain (higher for reverse)
car.throttle_gain = 0.25                                  # Change Throttle Gain here. Speed Will increase
print("✅ Both directions primed!")
#NOTE: '.1' works although very glitchy and stops before accelerating, should probably be above that (.15 worked well). 
# '.11' worked slightly better, more of a 70/30 messup rate 
# '.12' worked very well, only con is sometime the gain hits it and other times it doesn't, it's still slow but different speeds depending on if the gain hits or not
# '.13' worked like '.12' just almost always hit the gain so more consistent, this is a safer option
# Rarely when you full power accelerate it'll take time to remember what the accelerate and reverse does
# The strat for right now is do .3 before all

In [ ]:
#Only Run this if you want to stop the car 

In [ ]:
def emergency_stop(change):
    if change['new']:
        car.throttle = 0
        car.steering = 0
        print("🛑 EMERGENCY STOP!")

def reset_center(change):
    if change['new']:
        car.steering = 0
        print("🎯 Reset to center")

controller.buttons[0].observe(emergency_stop, names='value')
controller.buttons[1].observe(reset_center, names='value')

print("🛡️ Safety features active:")
print("   Button 0 (X/A) = Emergency stop")
print("   Button 1 (Circle/B) = Reset steering")